### Create Random Cert
---
The following python code segment gernerates 150 random certificates using the CA servers credentials.
**This is already run. No need to run again.**

In [ ]:
%%writefile create-random-cert.py


from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from datetime import datetime, timedelta
import os

from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.asymmetric import utils
from cryptography.hazmat.backends import default_backend
import random
import base64

import pymysql
import random
import uuid
import hashlib

# Database connection configuration
configM = {
    'user': 'zarinprm',
    'password': '833289612',  # Replace with your actual password
    'host': 'faure',
    'database': 'zarinprm',  # Replace with your actual database name
    'port': 3306,
    'charset': 'utf8mb4',
    'cursorclass': pymysql.cursors.DictCursor
}

def generate_random_bool():
   random_boolean = bool(random.getrandbits(1))
   return random_boolean


def get_id():
    unique_id = uuid.uuid4()
    hashed = hashlib.sha256(str(unique_id).encode()).hexdigest()
    return hashed[:5]



def insert_cert(private_key, cert, expStatus, revStatus):
    conn = pymysql.connect(**configM)

    # Create a cursor object to interact with MariaDB
    cursor = conn.cursor()

    # SQL query
    query = "INSERT INTO CA_CERT( CSR, CERTIFICATE, ISEXPIRED, ISREVOKED) VALUES (%s, %s, %s, %s)"

    # Execute the query
    cursor.execute(query, (private_key, cert, expStatus, revStatus))
    conn.commit()

    # Close the cursor and connection
    cursor.close()
    conn.close()


def generate_client_certificate(validity):
    # Generate private key for the client
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )

    # Create a CSR
    csr = x509.CertificateSigningRequestBuilder().subject_name(x509.Name([
          x509.NameAttribute(x509.NameOID.COMMON_NAME, "example.com"),])
          ).add_extension(
          x509.BasicConstraints(ca=False, path_length=None), critical=True,
           ).sign(private_key, hashes.SHA256(), default_backend())


    gpass = "secret_password"
    valid_from = datetime.utcnow()
    valid_until = valid_from + timedelta(days=validity)

    # Load the CA's public key
    ca_public_key_file = open("ca-public-key.pem", "rb")
    ca_public_key = x509.load_pem_x509_certificate( ca_public_key_file.read(), default_backend() )


    ca_private_key_file = open("ca-private-key.pem", "rb")
    ca_private_key = serialization.load_pem_private_key( ca_private_key_file.read(),
                                                     gpass.encode("utf-8"),
                                                     default_backend(),
)

    builder = (
        x509.CertificateBuilder()
        .subject_name(csr.subject)
        .issuer_name(ca_public_key.subject)
        .public_key(csr.public_key())
        .serial_number(x509.random_serial_number())
        .not_valid_before(valid_from)
        .not_valid_after(valid_until)
    )

    for extension in csr.extensions:
        builder = builder.add_extension(extension.value, extension.critical)

    public_key = builder.sign(
        private_key=ca_private_key,
        algorithm=hashes.SHA256(),
        backend=default_backend(),
    )

    cert_pem = public_key.public_bytes(serialization.Encoding.PEM)
    csr_pem = csr.public_bytes(serialization.Encoding.PEM)
    return csr_pem, cert_pem



# Generate certificates for 10 clients
for i in range(1, 151):
    boolExp = generate_random_bool()
    boolRev = generate_random_bool()

    #print(i, boolStatus, boolRevStatus)

    validity = 40
    expStatus = 'N'

    revStatus = 'N'

    if boolExp:
      validity = 1
      expStatus = 'Y'

    if boolRev:
      revStatus = 'Y'

    csr_pem, cert_pem = generate_client_certificate(validity)
    insert_cert(csr_pem, cert_pem, expStatus, revStatus)








Writing create-random-cert.py


### Repository Server

---


Repository server returna a list of Certificate Id and their content to verify.


In [ ]:
%%writefile reposittory-server.py



from cryptography import x509
from cryptography.hazmat.backends import default_backend
from getpass import getpass  # password reader
from cryptography.hazmat.primitives import serialization
from flask import Response, make_response
from cryptography.hazmat.primitives import serialization
from datetime import datetime, timedelta
from cryptography.hazmat.primitives import hashes

from flask import Flask, request, jsonify

import base64

import pymysql
import ast
import random

app = Flask(__name__)

configM = {
    'user': 'zarinprm',
    'password': '833289612',  # Replace with your actual password
    'host': 'faure',
    'database': 'zarinprm',  # Replace with your actual database name
    'port': 3306,
    'charset': 'utf8mb4',
    'cursorclass': pymysql.cursors.DictCursor
}


@app.route("/")
def get_secret_message():
    return 'Welcome to the certificate repository. You can get randomly generated certificate from this server to play around. Have fun!!!'


#############################################################################################
def get_student_history_by_id(student_id):

    conn = pymysql.connect(**configM)
    print("Connected to the MariaDB database!")

    # Create a cursor object to interact with MariaDB
    cursor = conn.cursor()

    # SQL query
    query = "SELECT * FROM STUDENT_HISTORY WHERE SID = %s"

    # Execute the query
    cursor.execute(query, (student_id))

    # Fetch one row
    row = cursor.fetchone()

    # Close the cursor and connection
    cursor.close()
    conn.close()

    return row

def generate_cert_id():
    numbers = random.sample(range(1, 101), 10)
    return numbers

def save_list_of_cert(student_id, cert_id, rev_map, exp_map):

    # Convert list to string
    cert_id_str = str(cert_id)
    rev_map_str = str(rev_map)
    exp_map_str = str(exp_map)

    conn = pymysql.connect(**configM)
    cursor = conn.cursor()

    # SQL query
    query = """
    UPDATE STUDENT_HISTORY
    SET CERT_LIST = %s, ISEXP= %s, ISREV=%s
    WHERE SID = %s
    """

    # Execute the query
    cursor.execute(query, (cert_id_str, exp_map_str, rev_map_str, student_id))

    # Commit the changes
    conn.commit()

    # Close the cursor and connection
    cursor.close()
    conn.close()


def prepare_answer_map(cert_list):
    rev_map = {}
    exp_map = {}

    for cert_id in cert_list:
       try:
          cert = get_certificate_by_id(cert_id)

          if cert:
             is_rev = cert['ISREVOKED']
             is_exp = cert['ISEXPIRED']

             rev_map[cert_id] = is_rev
             exp_map[cert_id] = is_exp

          else:
             return jsonify({"error": "No certificate found with the given id"}), 404

       except pymysql.MySQLError as err:
          print(f"Error: {err}")
          return 'Unexpected Error Occured. Please try again sometimes later.', 404

    return rev_map, exp_map


def prepare_response(cert_list):
    response = ' Certificate Id List:<br>'
    print(cert_list)
    for i in cert_list:
       response = response + '		CERT' + str(i) +'<br>'

    return response



@app.route('/get-cert-list')
def get_cert_list():

    sid = request.args.get('sid')

    try:

      student_history = get_student_history_by_id(sid)

      if student_history:

        cert_list_str= student_history['CERT_LIST']

        if cert_list_str:
            cert_list = ast.literal_eval(cert_list_str)



        else:
            cert_list = generate_cert_id()
            rev_map, exp_map = prepare_answer_map(cert_list)

            save_list_of_cert(sid, cert_list, rev_map, exp_map)

        return prepare_response(cert_list)



      else:
        return jsonify({"error": "No student found with the given credentials. Please check your Student Id."}), 404

    except pymysql.MySQLError as err:
      print(f"Error: {err}")
      return 'Unexpected Error Occured. Please try again sometimes later.', 404

#############################################################################################

def get_certificate_by_id(cert_id):

    conn = pymysql.connect(**configM)
    print("Connected to the MariaDB database!")

    # Create a cursor object to interact with MariaDB
    cursor = conn.cursor()

    # SQL query
    query = "SELECT * FROM CA_CERT WHERE ID = %s"

    # Execute the query
    cursor.execute(query, (cert_id))

    # Fetch one row
    row = cursor.fetchone()

    # Close the cursor and connection
    cursor.close()
    conn.close()

    return row


def valid_sid_and_cid_pair(sid, cert_id):
    try:
      student_history = get_student_history_by_id(sid)

      if student_history:
        cert_list_str= student_history['CERT_LIST']

        if cert_list_str:
            cert_list = ast.literal_eval(cert_list_str)

            if cert_id in cert_list:

               return True

    except pymysql.MySQLError as err:
      return False

    return False



@app.route('/get-certificate')
def get_cert():
    sid = request.args.get('sid')

    cert_id = request.args.get('cert_id')
    cert_id = int(cert_id.replace('CERT', ''))

    if (not cert_id or not sid):
      return jsonify({"error": "Certificate ID and Student ID are required!!!"}), 400

    if(not valid_sid_and_cid_pair(sid, cert_id)):
       return jsonify({"error": "Invalid Certificate and Student Id provided!!!"}), 400


    try:
      cert = get_certificate_by_id(cert_id)

      if cert:

        certificate = cert['CERTIFICATE']

        response = make_response(certificate)
        filename= 'CERT_'+str(cert_id)+'.pem'
        response.headers['Content-Type'] = 'application/x-pem-file'
        response.headers['Content-Disposition'] = f'attachment; filename={filename}'
        return response;
      else:
        return jsonify({"error": "No certificate found with the given id"}), 404

    except pymysql.MySQLError as err:
      print(f"Error: {err}")
      return 'Unexpected Error Occured. Please try again sometimes later.', 404


#############################################################################################

@app.route('/verify-revoked-status')
def verify_status():
    sid = request.args.get('sid')
    cert_id = request.args.get('cert_id')
    cert_id = int(cert_id.replace('CERT', ''))

    if (not cert_id or not sid):
      return jsonify({"error": "Certificate ID and Student ID are required!!!"}), 400

    if(not valid_sid_and_cid_pair(sid, cert_id)):
       return jsonify({"error": "Invalid Certificate and Student Id provided!!!"}), 400


    try:
      cert = get_certificate_by_id(cert_id)

      if cert:
        status= cert['ISREVOKED']
        sts = 'REVOKED'

        if status =='N':
           sts = 'Not REVOKED'

        response = 'Certificate Id: ' + str(cert_id) + '. Status: '+ sts
        return response
      else:
        return jsonify({"error": "No certificate found with the given id"}), 404

    except pymysql.MySQLError as err:
      print(f"Error: {err}")
      return 'Unexpected Error Occured. Please try again sometimes later.', 404

#############################################################################################
def get_student_exp_map(sid):
    student_history = get_student_history_by_id(sid)
    exp_map = {}

    if student_history:
       exp_map_str= student_history['ISEXP']
       exp_map = ast.literal_eval(exp_map_str)

    else:
        print('No studnet Hiatory found with id ', sid)

    return exp_map


def store_exp_answers(exp_ans_map, correct, student_id):
    # Convert list to string
    exp_ans_map_str = str(exp_ans_map)

    conn = pymysql.connect(**configM)
    cursor = conn.cursor()

    # SQL query
    query = """
    UPDATE STUDENT_HISTORY
    SET EXP_ANS = %s, EXP_COR= %s
    WHERE SID = %s
    """

    # Execute the query
    cursor.execute(query, (exp_ans_map_str, correct, student_id))

    # Commit the changes
    conn.commit()

    # Close the cursor and connection
    cursor.close()
    conn.close()


@app.route('/submit-answer/expired-status', methods=['POST'])
def submit_expired_cert_answer():
    sid = request.args.get('sid')
    my_map = request.get_json()

    if (sid is None or my_map is None):
        return 'API called with wrong parameters. Please check student id or the dictionary of answers.'

    exp_map = get_student_exp_map(sid)

    exp_ans_map = {}
    correct = 0;

    for key, value in my_map.items():
        ans_value = exp_map.get(int(key))

        if (value is not None and value==ans_value) :
            exp_ans_map[int(key)] = value
            correct = correct + 1


    store_exp_answers(exp_ans_map, correct, sid)
    return 'You have submitted your answers for expired status verification of certificates. You got ' + str(correct) + ' correct out of 10.'





#############################################################################################
def get_student_rev_map(sid):
    student_history = get_student_history_by_id(sid)
    rev_map = {}

    if student_history:
       rev_map_str= student_history['ISREV']
       rev_map = ast.literal_eval(rev_map_str)

    else:
        print('No studnet history found with Id ', sid)

    return rev_map


def store_rev_answers(rev_ans_map, correct, student_id):
    # Convert list to string
    rev_ans_map_str = str(rev_ans_map)

    conn = pymysql.connect(**configM)
    cursor = conn.cursor()

    # SQL query
    query = """
    UPDATE STUDENT_HISTORY
    SET REV_ANS = %s, REV_COR= %s
    WHERE SID = %s
    """

    # Execute the query
    cursor.execute(query, (rev_ans_map_str, correct, student_id))

    # Commit the changes
    conn.commit()

    # Close the cursor and connection
    cursor.close()
    conn.close()

@app.route('/submit-answer/revoked-status' , methods=['POST'])
def submit_revoked_cert_answer():
    sid = request.args.get('sid')
    my_map = request.get_json()

    if (sid is None or my_map is None):
        return 'API called with wrong parameters. Please check student id or the answer dictionary.'

    rev_map = get_student_rev_map(sid)

    rev_ans_map = {}
    correct = 0;

    for key, value in my_map.items():
        ans_value = rev_map.get(int(key))

        if (value is not None and value==ans_value) :
            rev_ans_map[int(key)] = value
            correct = correct + 1


    store_rev_answers(rev_ans_map, correct, sid)
    return 'You have submitted your answers for revoked status verification of certificates. You got ' + str(correct) + ' correct out of 10.'











###########################################################################################

if __name__ == "__main__":
    app.run(host="0.0.0.0", port= 10300, ssl_context=( 'server-public-key.pem', 'server-private-key.key') )  #ssl_context=('cert.crt', 'key.pem'))





Overwriting reposittory-server.py
